In [ ]:


import cudaq
import numpy as np
from math import floor
import matplotlib.pyplot as plt
import random
from collections import Counter
from scipy.optimize import minimize
import time

In [ ]:


def compute_energy(sequence):
    N = len(sequence)
    energy = 0
    for k in range(1, N):
        C_k = sum(sequence[i] * sequence[i + k] for i in range(N - k))
        energy += C_k ** 2
    return energy


def bitstring_to_sequence(bitstring):
    return [1 if b == '0' else -1 for b in bitstring]


def sequence_to_bitstring(sequence):
    return ''.join('0' if s == 1 else '1' for s in sequence)


def random_sequence(N):
    return [random.choice([1, -1]) for _ in range(N)]


def combine(parent1, parent2):
    child = []
    for i in range(len(parent1)):
        if random.random() < 0.5:
            child.append(parent1[i])
        else:
            child.append(parent2[i])
    return child


def mutate(sequence, p_mutate=0.1):
    mutated = sequence.copy()
    for i in range(len(mutated)):
        if random.random() < p_mutate:
            mutated[i] *= -1
    return mutated


def tabu_search(initial_sequence, max_iterations=100, tabu_tenure=7):
    N = len(initial_sequence)
    current = initial_sequence.copy()
    current_energy = compute_energy(current)
    
    best = current.copy()
    best_energy = current_energy
    
    tabu_list = {}
    
    for iteration in range(max_iterations):
        best_neighbor = None
        best_neighbor_energy = float('inf')
        best_move = None
        
        for i in range(N):
            neighbor = current.copy()
            neighbor[i] *= -1
            neighbor_energy = compute_energy(neighbor)
            
            is_tabu = i in tabu_list and tabu_list[i] > iteration
            
            if is_tabu and neighbor_energy >= best_energy:
                continue
            
            if neighbor_energy < best_neighbor_energy:
                best_neighbor = neighbor
                best_neighbor_energy = neighbor_energy
                best_move = i
        
        if best_neighbor is None:
            break
        
        current = best_neighbor
        current_energy = best_neighbor_energy
        tabu_list[best_move] = iteration + tabu_tenure
        
        if current_energy < best_energy:
            best = current.copy()
            best_energy = current_energy
    
    return best, best_energy


def memetic_tabu_search(N, population_size=20, max_generations=50, 
                        p_mutate=0.1, tabu_iterations=100, initial_population=None):
    if initial_population is not None:
        population = [seq.copy() for seq in initial_population]
    else:
        population = [random_sequence(N) for _ in range(population_size)]
    
    energies = [compute_energy(seq) for seq in population]
    
    best_idx = np.argmin(energies)
    best_sequence = population[best_idx].copy()
    best_energy = energies[best_idx]
    
    energy_history = [best_energy]
    
    for generation in range(max_generations):
        if random.random() < 0.5:
            parent1 = random.choice(population)
            parent2 = random.choice(population)
            child = combine(parent1, parent2)
        else:
            child = random.choice(population).copy()
        
        if random.random() < p_mutate:
            child = mutate(child, p_mutate=0.1)
        
        improved_child, child_energy = tabu_search(child, max_iterations=tabu_iterations)
        
        worst_idx = np.argmax(energies)
        if child_energy < energies[worst_idx]:
            population[worst_idx] = improved_child
            energies[worst_idx] = child_energy
        
        if child_energy < best_energy:
            best_sequence = improved_child.copy()
            best_energy = child_energy
        
        energy_history.append(best_energy)
    
    return best_sequence, best_energy, population, energy_history

In [ ]:

def get_interactions(N):
    G2_weighted = {}
    G4_weighted = {}
    constant = 0
    
    for k in range(1, N):
        pairs_for_k = [(i, i + k) for i in range(N - k)]
        num_pairs = len(pairs_for_k)
    
        constant += num_pairs
        
        for idx1 in range(num_pairs):
            for idx2 in range(idx1 + 1, num_pairs):
                i1, j1 = pairs_for_k[idx1]
                i2, j2 = pairs_for_k[idx2]
                
                indices = tuple(sorted([i1, j1, i2, j2]))
                unique_indices = set(indices)
                
                if len(unique_indices) == 4:
                    key = tuple(sorted([i1, j1, i2, j2]))
                    G4_weighted[key] = G4_weighted.get(key, 0) + 2
                elif len(unique_indices) == 3:
                    counts = Counter([i1, j1, i2, j2])
                    singles = sorted([idx for idx, cnt in counts.items() if cnt == 1])
                    key = tuple(singles)
                    G2_weighted[key] = G2_weighted.get(key, 0) + 2
                elif len(unique_indices) == 2:
                    constant += 2
    
    return G2_weighted, G4_weighted, constant


def verify_hamiltonian_encoding(N, num_samples=100):
    G2_weighted, G4_weighted, constant = get_interactions(N)
    
    all_match = True
    max_diff = 0
    
    for _ in range(num_samples):
        seq = random_sequence(N)
        classical_energy = compute_energy(seq)
        
        hamiltonian_energy = constant
        for (i, j), weight in G2_weighted.items():
            hamiltonian_energy += weight * seq[i] * seq[j]
        for (i, j, k, l), weight in G4_weighted.items():
            hamiltonian_energy += weight * seq[i] * seq[j] * seq[k] * seq[l]
        
        diff = abs(classical_energy - hamiltonian_energy)
        max_diff = max(max_diff, diff)
        if diff > 1e-10:
            all_match = False
    
    return all_match, max_diff



In [ ]:

def canonicalize_sequence(sequence):

    seq = list(sequence)
    variants = [
        seq,
        [-s for s in seq],
        seq[::-1],
        [-s for s in seq[::-1]]
    ]
    
    bitstrings = [sequence_to_bitstring(v) for v in variants]
    min_idx = np.argmin(bitstrings)
    
    return variants[min_idx], bitstrings[min_idx]


def canonicalize_bitstring(bitstring):
    seq = bitstring_to_sequence(bitstring)
    _, canonical_bs = canonicalize_sequence(seq)
    return canonical_bs


def compute_diversity_metrics(samples_dict):
    total_count = sum(samples_dict.values())
    num_unique_raw = len(samples_dict)
    
    probs_raw = np.array(list(samples_dict.values())) / total_count
    entropy_raw = -np.sum(probs_raw * np.log2(probs_raw + 1e-12))
    
    canonical_counts = {}
    for bs, count in samples_dict.items():
        canon = canonicalize_bitstring(bs)
        canonical_counts[canon] = canonical_counts.get(canon, 0) + count
    
    num_unique_canonical = len(canonical_counts)
    probs_canon = np.array(list(canonical_counts.values())) / total_count
    entropy_canonical = -np.sum(probs_canon * np.log2(probs_canon + 1e-12))
    
    return {
        'num_unique_raw': num_unique_raw,
        'num_unique_canonical': num_unique_canonical,
        'entropy_raw': entropy_raw,
        'entropy_canonical': entropy_canonical,
        'symmetry_collapse_ratio': num_unique_canonical / max(num_unique_raw, 1)
    }



In [ ]:

def compute_cvar(energies, counts, alpha=0.2):
   
    all_energies = []
    for e, c in zip(energies, counts):
        all_energies.extend([e] * c)
    
    all_energies = np.array(all_energies)
    total = len(all_energies)
    sorted_energies = np.sort(all_energies)
    k = max(1, int(alpha * total))
    
    return np.mean(sorted_energies[:k])


def compute_best_k_mean(energies, counts, k=10):
    sorted_pairs = sorted(zip(energies, counts), key=lambda x: x[0])
    best_energies = [e for e, c in sorted_pairs[:k]]
    return np.mean(best_energies) if best_energies else float('inf')


def compute_sample_statistics(energies, counts, alpha=0.2):
    all_energies = []
    for e, c in zip(energies, counts):
        all_energies.extend([e] * c)
    
    all_energies = np.array(all_energies)
    
    return {
        'mean': np.mean(all_energies),
        'median': np.median(all_energies),
        'min': np.min(all_energies),
        'max': np.max(all_energies),
        'std': np.std(all_energies),
        'cvar': compute_cvar(energies, counts, alpha),
        'best_10_mean': compute_best_k_mean(energies, counts, k=10),
        'num_unique': len(energies)
    }

In [ ]:

@cudaq.kernel
def qaoa_circuit(N: int, p: int, 
                          G2_indices: list[list[int]], G2_weights: list[float],
                          G4_indices: list[list[int]], G4_weights: list[float],
                          gammas: list[float], betas: list[float]):
  
    reg = cudaq.qvector(N)
    h(reg)
    
    for layer in range(p):
        gamma = gammas[layer]
        beta = betas[layer]
        
        for idx in range(len(G2_indices)):
            pair = G2_indices[idx]
            weight = G2_weights[idx]
            i = pair[0]
            j = pair[1]
            angle = 2.0 * gamma * weight
            x.ctrl(reg[i], reg[j])
            rz(angle, reg[j])
            x.ctrl(reg[i], reg[j])
        
        for idx in range(len(G4_indices)):
            quad = G4_indices[idx]
            weight = G4_weights[idx]
            i = quad[0]
            t = quad[1]
            k = quad[2]
            l = quad[3]
            angle = 2.0 * gamma * weight
            x.ctrl(reg[i], reg[t])
            x.ctrl(reg[t], reg[k])
            x.ctrl(reg[k], reg[l])
            rz(angle, reg[l])
            x.ctrl(reg[k], reg[l])
            x.ctrl(reg[t], reg[k])
            x.ctrl(reg[i], reg[t])
        
        for q in range(N):
            rx(2.0 * beta, reg[q])


def prepare_circuit_data(N):
    G2_weighted, G4_weighted, constant = get_interactions(N)
    
    G2_indices = [list(k) for k in G2_weighted.keys()]
    G2_weights = [float(v) for v in G2_weighted.values()]
    
    G4_indices = [list(k) for k in G4_weighted.keys()]
    G4_weights = [float(v) for v in G4_weighted.values()]
    
    return G2_indices, G2_weights, G4_indices, G4_weights, constant

for test_N in [5, 10, 15]:
    G2_i, G2_w, G4_i, G4_w, const = prepare_circuit_data(test_N)
    print(f"  N={test_N}: {len(G2_i)} 2-body, {len(G4_i)} 4-body, const={const}")

In [ ]:


class QAOAOptimizer:
    def __init__(self, N, p=1, shots=1000, alpha=0.2, objective='cvar', verbose=True):
        self.N = N
        self.p = p
        self.shots = shots
        self.alpha = alpha
        self.objective = objective
        self.verbose = verbose
        
        self.G2_indices, self.G2_weights, self.G4_indices, self.G4_weights, self.constant = \
            prepare_circuit_data(N)
        
        self.eval_count = 0
        self.history = []
        self.best_params = None
        self.best_objective = float('inf')
        self.all_samples = {}
    
    def evaluate(self, params, confirm_shots=None):
        self.eval_count += 1
        
        gammas = list(params[:self.p])
        betas = list(params[self.p:])
        shots_to_use = confirm_shots if confirm_shots else self.shots
        
        result = cudaq.sample(
            qaoa_circuit_weighted,
            self.N, self.p,
            self.G2_indices, self.G2_weights,
            self.G4_indices, self.G4_weights,
            gammas, betas,
            shots_count=shots_to_use
        )
        
        energies = []
        counts = []
        
        for bitstring in result:
            count = result[bitstring]
            sequence = bitstring_to_sequence(bitstring)
            energy = compute_energy(sequence)
            
            energies.append(energy)
            counts.append(count)
            
            if bitstring not in self.all_samples:
                self.all_samples[bitstring] = {'count': 0, 'energy': energy}
            self.all_samples[bitstring]['count'] += count
        
        stats = compute_sample_statistics(energies, counts, self.alpha)
        
        if self.objective == 'cvar':
            obj_value = stats['cvar']
        elif self.objective == 'best_k':
            obj_value = stats['best_10_mean']
        else:
            obj_value = stats['mean']
        
        if obj_value < self.best_objective:
            self.best_objective = obj_value
            self.best_params = params.copy()
        
        self.history.append((params.copy(), obj_value, stats))
        
        if self.verbose and self.eval_count % 10 == 0:
            print(f"    Eval {self.eval_count}: obj={obj_value:.2f}, "
                  f"mean={stats['mean']:.2f}, min={stats['min']}, CVaR={stats['cvar']:.2f}")
        
        return obj_value
    
    def optimize_two_stage(self, 
                           gamma_range=(-np.pi, np.pi),
                           beta_range=(-np.pi/2, np.pi/2),
                           coarse_grid=8, fine_grid=5,
                           top_k_refine=3, confirm_shots_multiplier=3):
        
        print(f"\n  Stage 1: Coarse grid ({coarse_grid}x{coarse_grid})")
        
        gammas = np.linspace(gamma_range[0], gamma_range[1], coarse_grid)
        betas = np.linspace(beta_range[0], beta_range[1], coarse_grid)
        gamma_step = gammas[1] - gammas[0] if len(gammas) > 1 else 0.5
        beta_step = betas[1] - betas[0] if len(betas) > 1 else 0.25
        
        coarse_results = []
        
        for i, gamma in enumerate(gammas):
            for beta in betas:
                params = np.array([gamma, beta])
                obj = self.evaluate(params)
                coarse_results.append((params.copy(), obj))
            
            if (i + 1) % (coarse_grid // 4) == 0:
                print(f"    Coarse progress: {(i + 1) / coarse_grid * 100:.0f}%")
        
        coarse_results.sort(key=lambda x: x[1])
        print(f"  Top-3 coarse: {[(f'({r[0][0]:.2f},{r[0][1]:.2f})', f'{r[1]:.1f}') for r in coarse_results[:3]]}")
        
        print(f"\n  Stage 2: Fine grid ({fine_grid}x{fine_grid}) around top-{top_k_refine}")
        
        fine_results = []
        
        for coarse_params, _ in coarse_results[:top_k_refine]:
            g_center, b_center = coarse_params
            
            g_fine = np.linspace(g_center - gamma_step/2, g_center + gamma_step/2, fine_grid)
            b_fine = np.linspace(b_center - beta_step/2, b_center + beta_step/2, fine_grid)
            
            for gamma in g_fine:
                for beta in b_fine:
                    params = np.array([gamma, beta])
                    obj = self.evaluate(params)
                    fine_results.append((params.copy(), obj))
        
        fine_results.sort(key=lambda x: x[1])
        
        print(f"\n  Stage 3: Confirming with {self.shots * confirm_shots_multiplier} shots")
        
        best_candidate = fine_results[0][0]
        confirmed_obj = self.evaluate(best_candidate, 
                                      confirm_shots=self.shots * confirm_shots_multiplier)
        
        print(f"  Confirmed: gamma={best_candidate[0]:.4f}, beta={best_candidate[1]:.4f}, obj={confirmed_obj:.2f}")
        
        return self.best_params, self.best_objective
    
    def extract_diverse_seeds(self, population_size, verbose=True):
        samples_dict = {bs: data['count'] for bs, data in self.all_samples.items()}
        metrics = compute_diversity_metrics(samples_dict)
        
        if verbose:
            print(f"\n  Diversity: {metrics['num_unique_raw']} raw -> {metrics['num_unique_canonical']} canonical")
        
        canonical_map = {}
        for bitstring, data in self.all_samples.items():
            seq = bitstring_to_sequence(bitstring)
            energy = data['energy']
            count = data['count']
            _, canon_bs = canonicalize_sequence(seq)
            
            if canon_bs not in canonical_map:
                canonical_map[canon_bs] = (seq, energy, count)
            else:
                existing = canonical_map[canon_bs]
                if count > existing[2] or (count == existing[2] and energy < existing[1]):
                    canonical_map[canon_bs] = (seq, energy, count)
        
        sorted_canonical = sorted(canonical_map.values(), key=lambda x: x[1])
        
        population = []
        energies = []
        
        for seq, energy, count in sorted_canonical:
            if len(population) >= population_size:
                break
            population.append(list(seq))
            energies.append(energy)
        
        num_from_qaoa = len(population)
        while len(population) < population_size:
            new_seq = random_sequence(self.N)
            population.append(new_seq)
            energies.append(compute_energy(new_seq))
        
        if verbose:
            print(f"  Seeds: {num_from_qaoa} from QAOA, {population_size - num_from_qaoa} random")
            print(f"  Energy range: [{min(energies)}, {max(energies)}], mean={np.mean(energies):.1f}")
        
        metrics['num_from_qaoa'] = num_from_qaoa
        return population, energies, metrics



In [ ]:


def qaoa_enhanced_mts(N, p=1, population_size=20, max_generations=30,
                               qaoa_shots=1000, tabu_iterations=50,
                               alpha=0.2, objective='cvar',
                               coarse_grid=8, fine_grid=5, verbose=True):
    results = {}
    
    print(f"\n{'='*70}")
    print(f"QAOA-Enhanced MTS for LABS (N={N}, p={p}, obj={objective})")
    print(f"{'='*70}")
    
    # Step 1: QAOA Optimization
    print(f"\n[Step 1] QAOA Parameter Optimization")
    
    qaoa = QAOAOptimizer(
        N=N, p=p, shots=qaoa_shots, alpha=alpha, objective=objective, verbose=verbose
    )
    
    start_time = time.time()
    best_params, best_obj = qaoa.optimize_two_stage(
        coarse_grid=coarse_grid, fine_grid=fine_grid, top_k_refine=3
    )
    qaoa_time = time.time() - start_time
    
    print(f"\n  QAOA done: {qaoa.eval_count} evals in {qaoa_time:.2f}s, best {objective}={best_obj:.2f}")
    
    results['qaoa_time'] = qaoa_time
    results['qaoa_best_obj'] = best_obj
    results['qaoa_best_params'] = best_params
    
    # Step 2: Extract Seeds
    print(f"\n[Step 2] Extracting Diverse Seeds")
    
    qaoa_population, qaoa_energies, diversity_metrics = qaoa.extract_diverse_seeds(
        population_size=population_size, verbose=verbose
    )
    
    results['diversity_metrics'] = diversity_metrics
    results['qaoa_init_energies'] = qaoa_energies.copy()
    
    # Step 3: QAOA-Seeded MTS
    print(f"\n[Step 3] Running MTS with QAOA Seeds")
    
    start_time = time.time()
    qaoa_best_seq, qaoa_best_energy, qaoa_final_pop, qaoa_history = memetic_tabu_search(
        N=N, population_size=population_size, max_generations=max_generations,
        p_mutate=0.3, tabu_iterations=tabu_iterations, initial_population=qaoa_population
    )
    qaoa_mts_time = time.time() - start_time
    
    print(f"  QAOA-MTS: best={qaoa_best_energy} in {qaoa_mts_time:.2f}s")
    
    results['qaoa_mts_time'] = qaoa_mts_time
    results['qaoa_best_energy'] = qaoa_best_energy
    results['qaoa_best_seq'] = qaoa_best_seq
    results['qaoa_history'] = qaoa_history
    results['qaoa_final_pop'] = qaoa_final_pop
    
    # Step 4: Baseline
    print(f"\n[Step 4] Running Baseline MTS")
    
    start_time = time.time()
    std_best_seq, std_best_energy, std_final_pop, std_history = memetic_tabu_search(
        N=N, population_size=population_size, max_generations=max_generations,
        p_mutate=0.3, tabu_iterations=tabu_iterations, initial_population=None
    )
    std_mts_time = time.time() - start_time
    
    print(f"  Standard MTS: best={std_best_energy} in {std_mts_time:.2f}s")
    
    results['std_mts_time'] = std_mts_time
    results['std_best_energy'] = std_best_energy
    results['std_best_seq'] = std_best_seq
    results['std_history'] = std_history
    results['std_final_pop'] = std_final_pop
    
    improvement = std_best_energy - qaoa_best_energy
    
    print(f"\n{'='*70}")
    print(f"RESULTS: QAOA-MTS={qaoa_best_energy}, Standard={std_best_energy}")
    print(f"Improvement: {improvement} {'(QAOA better)' if improvement > 0 else '(Tie or Random better)'}")
    print(f"{'='*70}")
    
    results['N'] = N
    results['p'] = p
    results['improvement'] = improvement
    
    return results


In [ ]:


N_experiment = 12
p_qaoa = 1

results = qaoa_enhanced_mts(
    N=N_experiment,
    p=p_qaoa,
    population_size=15,
    max_generations=25,
    qaoa_shots=800,
    tabu_iterations=40,
    alpha=0.2,
    objective='cvar',
    coarse_grid=8,
    fine_grid=5,
    verbose=True
)